**python -m venv ./venv**

Run that in your terminal to set up the virtual environment, then run the below command. If you don't care about using a virtual environment, just run the below command to install the required packages for your current version of Python.

In [ ]:
%pip install pandas numpy matplotlib seaborn scikit-learn tensorflow wordcloud

You can also run the following:

In [ ]:
%pip install -r requirements.txt

**Process 1: Data Cleaning**